# Computer Vision Homework 1

Question 3

Amirkabir University of Technology

Dr. Safabakhsh

By Gholamreza Dar 400131018

Fall 2022


## Imports

In [ ]:
import cv2
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns; sns.set(); sns.set_style('dark')

import os
import datetime

## Functions

In [ ]:
def rgb(img):
    return cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

def bgr(img):
    return cv2.cvtColor(img, cv2.COLOR_RGB2BGR)

def gray(img):
    return cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

def disp(img, title='', s=8, vmin=None, vmax=None):
    plt.figure(figsize=(s,s))
    plt.axis('off')
    if vmin is not None and vmax is not None:
        plt.imshow(img, cmap='gray', vmin=vmin, vmax=vmax)
    else:
        plt.imshow(img, cmap='gray')
    plt.title(title)
    plt.show()

def uint8ify(img):
    img -= img.min()
    img /= img.max()
    img *= 255
    return np.uint8(img)

def overlay(a,b):
    # a and b should be float images and between 0-1

    mask = a >= 0.5 # generate boolean mask of everywhere a > 0.5 
    ab = np.zeros_like(a) # generate an output container for the blended image 

    # now do the blending 
    ab[~mask] = (2*a*b)[~mask] # 2ab everywhere a<0.5
    ab[mask] = (1-2*(1-a)*(1-b))[mask] # else this
    
    return ab

def before_after(img_a, img_b, name='', vmin=None, vmax=None, effect_name='Processed'):
    fig, axs = plt.subplots(1,2, constrained_layout=True, figsize=(10,4))
    axs[0].axis('off')
    axs[1].axis('off')
    axs[0].set_title(f'{name} Original')
    axs[1].set_title(f'{name} {effect_name}')
    if vmin is not None and vmax is not None:
        axs[0].imshow(img_a, cmap='gray', vmin=vmin, vmax=vmax)
        axs[1].imshow(img_b, cmap='gray', vmin=vmin, vmax=vmax)
    else:
        axs[0].imshow(img_a, cmap='gray')
        axs[1].imshow(img_b, cmap='gray')
    plt.show()

# psnr function
import math
def psnr(img1, img2):
    mse = np.mean((img1 - img2) ** 2)
    if mse == 0:
        return 100
    PIXEL_MAX = 255.0
    return 20 * math.log10(PIXEL_MAX / math.sqrt(mse))

## Loading Images

In [ ]:
images = [cv2.imread(f"data/0{i}.jpg", 0) for i in range(1, 5)]
disp(images[0], 'Original 1', s=12)

## Part 1

In [ ]:
def extract_images(image):
    """Takes a Tall images and splits it vertically into three images

    Args:
        image (_type_): Tall image

    Returns:
        (image_r, image_g, image_b): a 3-tuple of images
    """

    h = image.shape[0]
    image_r = image[:h//3]
    image_g = image[h//3:2*h//3]
    image_b = image[2*h//3:]

    # Make the images the same height
    min_h = min(image_r.shape[0], image_g.shape[0], image_b.shape[0])
    image_r = image_r[:min_h]
    image_g = image_g[:min_h]
    image_b = image_b[:min_h]

    # Make sure the images are the same size
    assert image_r.shape == image_g.shape == image_b.shape, "Splitted images are not the same size"

    return image_r, image_g, image_b

In [ ]:
for image_id, image in enumerate(images):
    # Split the image into three channels
    image_r, image_g, image_b = extract_images(image)

    # Plot the 3 images
    fig, axs = plt.subplots(1,3, constrained_layout=True, figsize=(10,4))
    fig.suptitle(f'Extracted Images from 0{image_id+1}.jpg')
    for ax, image, title in zip(axs.ravel(), [image_r, image_g, image_b], ['Channel 0', 'Channel 1', 'Channel 2']):
        ax.axis('off')
        ax.set_title(title)
        ax.imshow(image, cmap='gray')
    plt.show()


## Part 2

In [ ]:
def combine_images(image_r, image_g, image_b, order=[0,1,2]):
    """Takes three images and combines them into one image putting one in each channel based on 'order'

    Args:
        image_r (single channel image): Red Channel
        image_g (single channel image): Green Channel
        image_b (single channel image): Blue Channel
        order (list, optional): order of channels. Defaults to [0,1,2].

    Returns:
        Combined image (3 Channel Image): Combined RGB image
    """

    # Combine the images into a single image
    image = np.stack([image_r, image_g, image_b], axis=2)
    # Reorder RGB Channels based on 'order'
    image = image[:,:,order]

    return image
    

In [ ]:
# RGB orders for each image
orders = [
    #R G B
    [2,1,0], # 01.jpg
    [2,1,0], # 02.jpg
    [2,1,0], # 03.jpg
    [2,1,0], # 03.jpg
]

for image_id, image in enumerate(images):
    # Split the image into three channels
    image_r, image_g, image_b = extract_images(image)

    # Combine the images into a single image
    combined_image = combine_images(image_r, image_g, image_b, order=orders[image_id])

    disp(combined_image, f'Combined Image for 0{image_id+1}.jpg', s=12)

## Part 3

In [ ]:
def align_channels(image_r, image_g, image_b, movements_horizontal=[0, 0, 0], movements_vertical=[0, 0, 0]):
    """Take 3 Channel images and move them vertically by 'movements' values

    Args:
        image_r (single channel image): Red Channel
        image_g (single channel image): Green Channel
        image_b (single channel image): Blue Channel
        movements_horizontal (list, optional): Amount of horizontal movement(rightwards!) for each channel. Defaults to [0, 0, 0].
        movements_vertical (list, optional): Amount of vertical movement(downwards) for each channel. Defaults to [0, 0, 0].

    Returns:
        (image_r, image_g, image_b): a 3-tuple of images
    """

    # Move the images vertically and horizontally by 'movements_vertical' and 'movements_horizontal' respectively
    image_r = np.roll(image_r, movements_horizontal[0], axis=1)
    image_r = np.roll(image_r, movements_vertical[0], axis=0)

    image_g = np.roll(image_g, movements_horizontal[1], axis=1)
    image_g = np.roll(image_g, movements_vertical[1], axis=0)

    image_b = np.roll(image_b, movements_horizontal[2], axis=1)
    image_b = np.roll(image_b, movements_vertical[2], axis=0)
    
    return image_r, image_g, image_b

Let's try two methods for moving images vertically

In [ ]:
image_r_copy = image_r.copy()

In [ ]:
%%timeit
image_r_rolled = np.roll(image_r_copy, 10, axis=0)

In [ ]:
%%timeit
image_r_copy[10:, :] = image_r_copy[:-10, :]

This is faster so we use it!

Update: The second method has some bugs so we use the first one

In [ ]:
# The reason 0 indexing is and exception
a = [1,2,3,4,5,6,7,8,9,10]
print(a[0:])
print(a[:-0])

### Interactive Aligner

In [ ]:
from ipywidgets import interact

# Change this to try other images
IMAGE_ID = 0

@interact(rx=(-25, 25), ry=(-25, 25), gx=(-25, 25), gy=(-25, 25), bx=(-25, 25), by=(-25, 25))
def move_channels_interactive(rx=0,ry=0, gx=0, gy=0, bx=0, by=0):
    image=images[IMAGE_ID].copy()
    image_r, image_g, image_b = extract_images(image)

    # 2. align the images vertically
    movements_horizontal = [rx, gx, bx]
    movements_vertical = [ry, gy, by]
    image_r, image_g, image_b = align_channels(image_r, image_g, image_b, movements_horizontal, movements_vertical)

    # Combine the images into a single image (order=[2, 1, 0] is fixed this time)
    combined_image = combine_images(image_r, image_g, image_b, order=[2, 1, 0])
    
    # Calculate the PSNR between the resulting(aligned) grayscale image and the green channel(unchanged) of the image
    combined_image_gray = cv2.cvtColor(combined_image, cv2.COLOR_RGB2GRAY)
    # Ignore the border pixels
    p = 50 # padding
    psnr_value = cv2.PSNR(combined_image_gray[p:-p, p:-p], image_g[p:-p, p:-p])

    disp(combined_image, f'Combined+Aligned Image for 0{image_id+1}.jpg | PSNR={psnr_value}', s=9)

In [ ]:
from ipywidgets import interact

# Change this to try other images
IMAGE_ID = 1

@interact(rx=(-25, 25), ry=(-25, 25), gx=(-25, 25), gy=(-25, 25), bx=(-25, 25), by=(-25, 25))
def move_channels_interactive(rx=0,ry=0, gx=0, gy=0, bx=0, by=0):
    image=images[IMAGE_ID].copy()
    image_r, image_g, image_b = extract_images(image)

    # 2. align the images vertically
    movements_horizontal = [rx, gx, bx]
    movements_vertical = [ry, gy, by]
    image_r, image_g, image_b = align_channels(image_r, image_g, image_b, movements_horizontal, movements_vertical)

    # Combine the images into a single image (order=[2, 1, 0] is fixed this time)
    combined_image = combine_images(image_r, image_g, image_b, order=[2, 1, 0])

    # Calculate the PSNR between the resulting(aligned) grayscale image and the green channel(unchanged) of the image
    combined_image_gray = cv2.cvtColor(combined_image, cv2.COLOR_RGB2GRAY)
    # Ignore the border pixels
    p = 50 # padding
    psnr_value = cv2.PSNR(combined_image_gray[p:-p, p:-p], image_g[p:-p, p:-p])

    disp(combined_image, f'Combined+Aligned Image for 0{image_id+1}.jpg | PSNR={psnr_value}', s=9)

In [ ]:
from ipywidgets import interact

# Change this to try other images
IMAGE_ID = 2

@interact(rx=(-25, 25), ry=(-25, 25), gx=(-25, 25), gy=(-25, 25), bx=(-25, 25), by=(-25, 25))
def move_channels_interactive(rx=0,ry=0, gx=0, gy=0, bx=0, by=0):
    image=images[IMAGE_ID].copy()
    image_r, image_g, image_b = extract_images(image)

    # 2. align the images vertically
    movements_horizontal = [rx, gx, bx]
    movements_vertical = [ry, gy, by]
    image_r, image_g, image_b = align_channels(image_r, image_g, image_b, movements_horizontal, movements_vertical)

    # Combine the images into a single image (order=[2, 1, 0] is fixed this time)
    combined_image = combine_images(image_r, image_g, image_b, order=[2, 1, 0])

    # Calculate the PSNR between the resulting(aligned) grayscale image and the green channel(unchanged) of the image
    combined_image_gray = cv2.cvtColor(combined_image, cv2.COLOR_RGB2GRAY)
    # Ignore the border pixels
    p = 50 # padding
    psnr_value = cv2.PSNR(combined_image_gray[p:-p, p:-p], image_g[p:-p, p:-p])

    disp(combined_image, f'Combined+Aligned Image for 0{image_id+1}.jpg | PSNR={psnr_value}', s=9)

In [ ]:
from ipywidgets import interact

# Change this to try other images
IMAGE_ID = 3

@interact(rx=(-25, 25), ry=(-25, 25), gx=(-25, 25), gy=(-25, 25), bx=(-25, 25), by=(-25, 25))
def move_channels_interactive(rx=0,ry=0, gx=0, gy=0, bx=0, by=0):
    image=images[IMAGE_ID].copy()
    image_r, image_g, image_b = extract_images(image)

    # 2. align the images vertically
    movements_horizontal = [rx, gx, bx]
    movements_vertical = [ry, gy, by]
    image_r, image_g, image_b = align_channels(image_r, image_g, image_b, movements_horizontal, movements_vertical)

    # Combine the images into a single image (order=[2, 1, 0] is fixed this time)
    combined_image = combine_images(image_r, image_g, image_b, order=[2, 1, 0])

    # Calculate the PSNR between the resulting(aligned) grayscale image and the green channel(unchanged) of the image
    combined_image_gray = cv2.cvtColor(combined_image, cv2.COLOR_RGB2GRAY)
    # Ignore the border pixels
    p = 50 # padding
    psnr_value = cv2.PSNR(combined_image_gray[p:-p, p:-p], image_g[p:-p, p:-p])

    disp(combined_image, f'Combined+Aligned Image for 0{image_id+1}.jpg | PSNR={psnr_value}', s=9)

### Manually Found Alignments

In [ ]:
final_images = []
for image_id, image in enumerate(images):
    # 1. Split the image into three channels
    image_r, image_g, image_b = extract_images(image)

    # 2. align the images
    movements = [
        #rx gx bx    ry gy by
        [[0, 0, 0],  [-9, 0, 4]],  # 01.jpg
        [[0, 0, -1], [-6, 0, 6]], # 02.jpg
        [[0, 0, -1], [3, 0, 1]],  # 03.jpg
        [[0, 0, 0],  [-6, 0, 9]],  # 04.jpg
    ]
    image_r, image_g, image_b = align_channels(
        image_r, image_g, image_b, 
        movements_horizontal=movements[image_id][0], 
        movements_vertical=movements[image_id][1])

    # Combine the images into a single image (order=[2, 1, 0] is fixed this time)
    combined_image = combine_images(image_r, image_g, image_b, order=[2, 1, 0])
    final_images.append(combined_image)

    # Calculate the PSNR between the resulting(aligned) grayscale image and the green channel(unchanged) of the image
    combined_image_gray = cv2.cvtColor(combined_image, cv2.COLOR_RGB2GRAY)
    # Ignore the border pixels
    p = 50 # padding
    psnr_value = cv2.PSNR(combined_image_gray[p:-p, p:-p], image_g[p:-p, p:-p])

    disp(combined_image, f'Combined+Aligned Image for 0{image_id+1}.jpg | PSNR={psnr_value}', s=9)

### Crop Borders of combined images

In [ ]:
def crop_borders(image, padding=25):
    return image[padding:-padding, padding:-padding]

disp(crop_borders(final_images[0]), '01.jpg Combined & Cropped', s=9)
disp(crop_borders(final_images[1]), '02.jpg Combined & Cropped', s=9)
disp(crop_borders(final_images[2]), '03.jpg Combined & Cropped', s=9)
disp(crop_borders(final_images[3]), '04.jpg Combined & Cropped', s=9)

### Automatically find the best alignment

The alignment with the most psnr score

#### Naiive Method

In [ ]:
def extract_and_auto_align_naiive(image, image_id, ranges):
    # 1. Split the image into three channels
    image_r, image_g, image_b = extract_images(image)

    best_psnr_value = 0
    best_combined_image = None
    best_movements = [0, 0, 0, 0]
    i = 0
    
    total_states = (ranges[0][1] - ranges[0][0] + 1) * \
        (ranges[1][1] - ranges[1][0] + 1) * \
        (ranges[2][1] - ranges[2][0] + 1) * \
        (ranges[3][1] - ranges[3][0] + 1)

    # Grid search over the movements
    for ry in range(ranges[0][0], ranges[0][1]+1):
        for by in range(ranges[1][0], ranges[1][1]+1):
            for rx in range(ranges[2][0], ranges[2][1]+1):
                for bx in range(ranges[3][0], ranges[3][1]+1):
                    # 2. align the images
                    image_r_aligned, image_g_aligned, image_b_aligned = align_channels(
                        image_r, image_g, image_b, 
                        movements_horizontal=[rx, 0, bx], 
                        movements_vertical=[ry, 0, by])

                    # Combine the images into a single image (order=[2, 1, 0] is fixed this time)
                    combined_image = combine_images(image_r_aligned, image_g_aligned, image_b_aligned, order=[2, 1, 0])

                    # Calculate the PSNR between the resulting(aligned) grayscale image and the green channel(unchanged) of the image
                    combined_image_gray = cv2.cvtColor(combined_image, cv2.COLOR_RGB2GRAY)
                    # Ignore the border pixels
                    p = 50
                    psnr_value = cv2.PSNR(combined_image_gray[p:-p, p:-p], image_g[p:-p, p:-p])

                    if psnr_value > best_psnr_value:
                        best_psnr_value = psnr_value
                        best_combined_image = combined_image.copy()
                        best_movements = [rx, ry, bx, by]
                    
                    i += 1
                    if i%(total_states//20) == 0:
                        # disp(combined_image)
                        print(f"> Searching {i}/{total_states} ({i/total_states*100:.2f}%) | Best PSNR: {best_psnr_value} | Best Movements: {best_movements} ")
    disp(best_combined_image, title=f"0{image_id+1}.jpg | Best PSNR={best_psnr_value:.3f} | Best Movements={best_movements}")

In [ ]:
%%time
#             ry       by      rx       bx
ranges = [[-10, 10], [-10, 10], [-1, 1], [-1, 1]]
extract_and_auto_align_naiive(images[0], 0, ranges)

#### Faster Method

In [ ]:
def extract_and_auto_align_better(image, image_id, ranges, search_direction, found_vertical_movements):
    # 1. Split the image into three channels
    image_r, image_g, image_b = extract_images(image)

    best_psnr_value = 0
    best_combined_image = None
    best_movements = [0, 0, 0, 0]
    i = 0

    if search_direction == "vertical":
        total_states = (ranges[0][1] - ranges[0][0] + 1) * (ranges[1][1] - ranges[1][0] + 1)
        # Grid search over the movements(vertical)
        for ry in range(ranges[0][0], ranges[0][1]+1):
            for by in range(ranges[1][0], ranges[1][1]+1):
                # 2. align the images
                image_r_aligned, image_g_aligned, image_b_aligned = align_channels(
                    image_r, image_g, image_b, 
                    movements_horizontal=[0, 0, 0], # no horizontal movement for now
                    movements_vertical=[ry, 0, by]) # only try vertical movements

                # Combine the images into a single image (order=[2, 1, 0] is fixed this time)
                combined_image = combine_images(image_r_aligned, image_g_aligned, image_b_aligned, order=[2, 1, 0])

                # Calculate the PSNR between the resulting(aligned) grayscale image and the green channel(unchanged) of the image
                combined_image_gray = cv2.cvtColor(combined_image, cv2.COLOR_RGB2GRAY)
                # Ignore the border pixels
                p = 50
                psnr_value = cv2.PSNR(combined_image_gray[p:-p, p:-p], image_g[p:-p, p:-p])

                if psnr_value > best_psnr_value:
                    best_psnr_value = psnr_value
                    best_combined_image = combined_image.copy()
                    best_movements = [0, ry, 0, by]
                
                i += 1
                if i%(total_states//10) == 0:
                    print(f"> Searching (vertically) {i}/{total_states} ({i/total_states*100:.2f}%) | Best PSNR: {best_psnr_value:.3f} | Best Movements: {best_movements} ")
        
    if search_direction == "horizontal":
        # Grid search over the movements(horizontal)
        for rx in range(ranges[2][0], ranges[2][1]+1):
            for bx in range(ranges[3][0], ranges[3][1]+1):
                total_states = (ranges[2][1] - ranges[2][0] + 1) * (ranges[3][1] - ranges[3][0] + 1)
                locked_r_y, locked_b_y = found_vertical_movements
                # 2. align the images
                image_r_aligned, image_g_aligned, image_b_aligned = align_channels(
                    image_r, image_g, image_b, 
                    movements_horizontal=[rx, 0, bx], # tune the horizontal movement
                    movements_vertical=[locked_r_y, 0, locked_b_y]) # use locked values for vertical movement

                # Combine the images into a single image (order=[2, 1, 0] is fixed this time)
                combined_image = combine_images(image_r_aligned, image_g_aligned, image_b_aligned, order=[2, 1, 0])

                # Calculate the PSNR between the resulting(aligned) grayscale image and the green channel(unchanged) of the image
                combined_image_gray = cv2.cvtColor(combined_image, cv2.COLOR_RGB2GRAY)
                # Ignore the border pixels
                p = 50
                psnr_value = cv2.PSNR(combined_image_gray[p:-p, p:-p], image_g[p:-p, p:-p])

                if psnr_value > best_psnr_value:
                    best_psnr_value = psnr_value
                    best_combined_image = combined_image.copy()
                    best_movements = [rx, locked_r_y, bx, locked_b_y]
                
                i += 1
                if i%(total_states//10+1) == 0:
                    print(f"> Searching (horizontally) {i}/{total_states} ({i/total_states*100:.2f}%) | Best PSNR: {best_psnr_value} | Best Movements: {best_movements} ")

    disp(best_combined_image, title=f"0{image_id+1}.jpg | Best PSNR={best_psnr_value:.3f} | Best Movements={best_movements}")

    # return r_y and b_y to be locked in the horizontal search
    if search_direction == "vertical":
                #      r_y                b_y
        return [best_movements[1], best_movements[3]]
    # return best combined image after horizontal searach
    if search_direction == "horizontal":
                #      r_y                b_y
        return best_combined_image

In [ ]:
%%time
#             ry       by      rx       bx
ranges = [[-10, 10], [-10, 10], [-1, 1], [-1, 1]]
found_vertical_movements = extract_and_auto_align_better(images[0], 0, ranges, search_direction="vertical", found_vertical_movements=None)

In [ ]:
%%time
#             ry       by      rx       bx
ranges = [[-10, 10], [-10, 10], [-1, 1], [-1, 1]]
#                                                                                                                  [r_y, b_y]
extract_and_auto_align_better(images[0], 0, ranges, search_direction="horizontal", found_vertical_movements=found_vertical_movements)

Results
- Naiive method: 3969 checks (~7.6 secs)
- Direction seperated method: 441+9 checks (~2 secs)

In [ ]:
def extract_and_auto_align_better_wrapper(image, image_id, ranges):
    found_vertical_movements = extract_and_auto_align_better(image, image_id, ranges, search_direction="vertical", found_vertical_movements=None)
    return extract_and_auto_align_better(image, image_id, ranges, search_direction="horizontal", found_vertical_movements=found_vertical_movements)

#### Automatic Alignment for Images

In [ ]:
%%time
final_images_auto = []
#             ry       by      rx       bx
ranges = [[-10, 10], [-10, 10], [-1, 1], [-1, 1]]
for i in range(len(images)):
    print(f">> Image {i+1}")
    final_image = extract_and_auto_align_better_wrapper(images[i], image_id=i, ranges=ranges)
    final_images_auto.append(final_image)
    print()

In [ ]:
disp(crop_borders(final_images_auto[0]), '01.jpg Auto Combined & Cropped', s=9)
disp(crop_borders(final_images_auto[1]), '02.jpg Auto Combined & Cropped', s=9)
disp(crop_borders(final_images_auto[2]), '03.jpg Auto Combined & Cropped', s=9)
disp(crop_borders(final_images_auto[3]), '04.jpg Auto Combined & Cropped', s=9)

### Numba experiments

In [ ]:
from numba import jit

@jit
def extract_and_auto_align_naiive_jit(image, image_id, ranges):
    # 1. Split the image into three channels
    image_r, image_g, image_b = extract_images(image)

    best_psnr_value = 0
    best_combined_image = None
    best_movements = [0, 0, 0, 0]
    i = 0
    
    total_states = (ranges[0][1] - ranges[0][0] + 1) * \
        (ranges[1][1] - ranges[1][0] + 1) * \
        (ranges[2][1] - ranges[2][0] + 1) * \
        (ranges[3][1] - ranges[3][0] + 1)

    # Grid search over the movements
    for ry in range(ranges[0][0], ranges[0][1]+1):
        for by in range(ranges[1][0], ranges[1][1]+1):
            for rx in range(ranges[2][0], ranges[2][1]+1):
                for bx in range(ranges[3][0], ranges[3][1]+1):
                    # 2. align the images
                    image_r_aligned, image_g_aligned, image_b_aligned = align_channels(
                        image_r, image_g, image_b, 
                        movements_horizontal=[rx, 0, bx], 
                        movements_vertical=[ry, 0, by])

                    # Combine the images into a single image (order=[2, 1, 0] is fixed this time)
                    combined_image = combine_images(image_r_aligned, image_g_aligned, image_b_aligned, order=[2, 1, 0])

                    # Calculate the PSNR between the resulting(aligned) grayscale image and the green channel(unchanged) of the image
                    combined_image_gray = cv2.cvtColor(combined_image, cv2.COLOR_RGB2GRAY)
                    # Ignore the border pixels
                    p = 50
                    psnr_value = cv2.PSNR(combined_image_gray[p:-p, p:-p], image_g[p:-p, p:-p])

                    if psnr_value > best_psnr_value:
                        best_psnr_value = psnr_value
                        best_combined_image = combined_image.copy()
                        best_movements = [rx, ry, bx, by]
                    
                    i += 1
                    if i%(total_states//20) == 0:
                        # disp(combined_image)
                        # print(f"> Searching {i}/{total_states} ({i/total_states*100:.2f}%) | Best PSNR: {best_psnr_value} | Best Movements: {best_movements} ")
                        # print("> Searching {i}/{total_states} ({percent:.2f}%) | Best PSNR: {best_psnr_value} | Best Movements: {best_movements} ".format(i=i, total_states=total_states, percent=(i/total_states*100), best_psnr_value=best_psnr_value, best_movements=best_movements))
                        print()
    disp(best_combined_image, title="0{name}.jpg | Best PSNR={best_psnr_value:.3f} | Best Movements={best_movements}".format(name=image_id+1, best_psnr_value=best_psnr_value, best_movements=best_movements))

### Naiive non-jit

In [ ]:
%%time
#             ry       by      rx       bx
ranges = [[-10, 10], [-10, 10], [-1, 1], [-1, 1]]
extract_and_auto_align_naiive(images[0], 0, ranges)

### Naiive jit

In [ ]:
%%time
#             ry       by      rx       bx
ranges = [[-10, 10], [-10, 10], [-1, 1], [-1, 1]]
extract_and_auto_align_naiive_jit(images[0], 0, ranges)